In [8]:
# Step 1: Get data from OpenBB

# Start by importing pandas, the OpenBB SDK, QuantStats, and Backtrader.

import datetime as dt

import pandas as pd

from openbb_terminal.sdk import openbb
import quantstats as qs
import backtrader as bt
# There’s an unsolved issue with Backtrader that prevents it from downloading data. Here’s a simple workaround.

def openbb_data_to_bt_data(symbol, start_date, end_date):

    df = openbb.stocks.load(symbol, start_date=start_date, end_date=end_date)

    fn = f"data/{symbol.lower()}.csv"
    df.to_csv(fn)

    return bt.feeds.YahooFinanceCSVData(
        dataname=fn,
        fromdate=dt.datetime.strptime(start_date, '%Y-%m-%d'),
        todate=dt.datetime.strptime(end_date, '%Y-%m-%d')
    )
# This function downloads the data from the OpenBB SDK, converts it to a CSV, and reads it into Backtrader’s YahooFinanceCSVData.

In [3]:
# Step 2: Build a backtest using Backtrader

# Fund managers report their holdings every month. They don’t want to tell investors they lost money on the latest meme stock. So they sell them before the reporting deadline and buy higher-quality assets, like bonds.

# You might be able to take advantage of this effect by buying bonds toward the end of the month and selling them at the beginning.

# Start with a simple helper function that gets the last day of the month.

def last_day_of_month(any_day):
    # The day 28 exists in every month. 4 days later, it's always next month
    next_month = any_day.replace(day=28) + dt.timedelta(days=4)

    # subtracting the number of the current day brings us back one month
    return (next_month - dt.timedelta(days=next_month.day)).day
# Next, setup the Backtrader strategy. All Backtrader strategies are built as classes and inherit bt.Strategy.

In [4]:
class MonthlyFlows(bt.Strategy):

    params = (
        ("end_of_month", 23),
        ("start_of_month", 7),
    )

    def __init__(self):
        self.order = None
        self.dataclose = self.datas[0].close

    def notify_order(self, order):
        # No more orders
        self.order = None

    def next(self):

        # Get today's date, day of month, and last day of current month
        dt_ = self.datas[0].datetime.date(0)
        dom = dt_.day
        ldm = last_day_of_month(dt_)

        # If an order is pending, exit
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # We're in the first week of the month, sell
            if dom <= self.params.start_of_month:

                # Sell the entire portfolio
                self.order = self.order_target_percent(target=-1)

                print(f"Created SELL of {self.order.size} at {self.data_close[0]} on day {dom}")

            # We're in the last week of the month, buy
            if dom >= self.params.end_of_month:

                # Buy the entire portfolio
                self.order = self.order_target_percent(target=1)

                print(f"Created BUY of {self.order.size} {self.data_close[0]} on day {dom}")

        # We are not in the market
        else:

            # If we're long
            if self.position.size > 0:

                # And not within the last week of the month, close
                if not self.params.end_of_month <= dom <= ldm:

                    print(f"Created CLOSE of {self.position.size} at {self.data_close[0]} on day {dom}")

                    self.order = self.order_target_percent(target=0.0)

            # If we're short
            if self.position.size < 0:

                # And not within the first week of the month, close
                if not 1 <= dom <= self.params.start_of_month:

                    print(f"Created CLOSE of {self.position.size} at {self.data_close[0]} on day {dom}")

                    # self.order = self.close()
                    self.order = self.order_target_percent(target=0.0)

First, set up the strategy parameters. end_of_month is the first day I want to be long TLT and start_of_month is the last day I want to be short.

The strategy logic is in the next method.

This code tests if there’s a position in the market. If not, it checks if the current day is within the first week of the month and creates a short position. Otherwise, if the current day is within the last week, it creates a long position.

The last step closes the open positions.

The strategy closes the long position if the current day is no longer within the last week of the month. The strategy closes the short position if the current day is no longer within the first week of the month.

Now, run the backtest.

In [5]:
data = openbb_data_to_bt_data(
    "TLT",
    start_date="2002-01-01",
    end_date="2022-06-30"
)

cerebro = bt.Cerebro(stdstats=False)

cerebro.adddata(data)
cerebro.broker.setcash(1000.0)

cerebro.addstrategy(MonthlyFlows)

cerebro.addobserver(bt.observers.Value)

cerebro.addanalyzer(
    bt.analyzers.Returns, _name="returns"
)
cerebro.addanalyzer(
    bt.analyzers.TimeReturn, _name="time_return"
)

backtest_result = cerebro.run()

Loading Daily data for TLT with starting period 2002-07-30.

Created BUY of 24 40.82 on day 30
Created CLOSE of 24 at 41.56 on day 1
Created SELL of -24 at 41.99 on day 2
Created CLOSE of -24 at 41.91 on day 8
Created BUY of 23 43.02 on day 23
Created CLOSE of 23 at 44.28 on day 3
Created SELL of -23 at 44.27 on day 4
Created CLOSE of -23 at 43.92 on day 9
Created BUY of 23 45.31 on day 23
Created CLOSE of 23 at 44.9 on day 1
Created SELL of -23 at 45.16 on day 2
Created CLOSE of -23 at 45.28 on day 8
Created BUY of 24 42.57 on day 23
Created CLOSE of 24 at 43.6 on day 1
Created SELL of -24 at 43.41 on day 4
Created CLOSE of -24 at 44.85 on day 8
Created BUY of 23 43.55 on day 25
Created CLOSE of 23 at 43.42 on day 2
Created SELL of -23 at 43.48 on day 3
Created CLOSE of -23 at 44.14 on day 9
Created BUY of 22 44.52 on day 23
Created CLOSE of 22 at 44.17 on day 2
Created SELL of -22 at 44.28 on day 3
Created CLOSE of -22 at 44.54 on day 8
Created BUY of 22 44.77 on day 23
Created CLOSE of 22 at 45.15 on day 3
Created SELL of -22 at 45.35 on day 

In [6]:
# Get the strategy returns as a dictionary
returns_dict = backtest_result[0].analyzers.time_return.get_analysis()

# Convert the dictionary to a DataFrame
returns_df = (
    pd.DataFrame(
        list(returns_dict.items()),
        columns = ["date", "return"]
    )
    .set_index("date")
)

In [7]:
# Step 3: Assess the results using key performance metrics

# Trading takes time, money, and effort. To make sure you’re better off not being long TLT, compare the strategy results to a long-only strategy.

# QuantStats makes it easy.

# QuantStats is a library jam-packed with trading performance and risk metrics. One of the best parts of Backtrader is that it works well with QuantStats.

# Here’s how to use it.

bench = openbb.stocks.load(
    "TLT",
    start_date="2002-01-01",
    end_date="2022-06-30"
)["Adj Close"]

qs.reports.metrics(
    returns_df,
    benchmark=bench,
    mode="full"
)

Loading Daily data for TLT with starting period 2002-07-30.

                           Strategy     Benchmark
-------------------------  -----------  -----------
Start Period               2002-07-30   2002-07-30
End Period                 2022-06-29   2022-06-29
Risk-Free Rate             0.0%         0.0%
Time in Market             55.0%        4.0%

Cumulative Return          112.88%      -100.0%
CAGR﹪                     3.86%        -100.0%

Sharpe                     0.47         -0.2
Prob. Sharpe Ratio         98.13%       5.6%
Smart Sharpe               0.45         -0.19
Sortino                    0.67         -0.2
Smart Sortino              0.65         -0.19
Sortino/√2                 0.48         -0.14
Smart Sortino/√2           0.46         -0.14
Omega                      1.11         1.11

Max Drawdown               -27.79%      -100.0%
Longest DD Days            2586         7048
Volatility (ann.)          8.98%        22.53%
R^2                        0.0          0.0
Information Ratio          0.02         0.02
Calmar         